## Para iniciar as análises e preparação para a modelagem, vamos entender e preparar os datasets para criar uma base de treino e teste para os modelos preditivos.

### Começamos improtando todas as bibliotecas necessárias para esse trabalho.

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Vamos retirar a restrição de limite de colunas visíveis para ficar mais fácil visualizar as features.
pd.set_option('display.max_columns', None)

In [ ]:
# Vamos abrir as tabelas .csv extraídas da base de dados do app como dataframes de trabalho.
users = pd.read_csv('../CSVs/tbl_users.csv', sep=';', low_memory=False)
events = pd.read_csv('../CSVs/tbl_events.csv', low_memory=False)
users_events = pd.read_csv('../CSVs/tbl_users_events.csv', low_memory=False)
analise = pd.read_csv('../CSVs/gg_analise_db1.csv', sep=';', low_memory=False)
users_games = pd.read_csv('../CSVs/tbl_users_games.csv', low_memory=False)
games = pd.read_json('../CSVs/tbl_games.json')
friends = pd.read_csv('../CSVs/tbl_friends.csv', low_memory=False)
chats = pd.read_csv('../CSVs/tbl_chat_members.csv', low_memory=False)
plataformas = pd.read_csv('../CSVs/tbl_platforms.csv', low_memory=False)
platforms = pd.read_csv('../CSVs/tbl_users_platforms.csv', low_memory=False)
logs = pd.read_csv('../CSVs/tbl_users_login_log.csv', low_memory=False)

### Vamos começar o trabalho de organização, limpeza e engenharia de features.

In [ ]:
# Ao tentar fazer a mudança para INT, apareceram erros devido a alguns dados da coluna não serem numéricos.
# Aí eu busquei esses registros, que eram poucos, para eliminar as linhas.
users.loc[users['id']=='   "']
users.loc[users['id']=='?>"']

# Eliminando as linhas problemáticas.
users.drop(index=16790, inplace=True)
users.drop(index=22557, inplace=True)

# Aí, sim, alterando a coluna id para INT.
users.id = users.id.astype(int)

In [ ]:
# Definindo o ID como index e renomeando para 'user_id'.
gg_users = users.set_index('id', drop=True)
gg_users.rename_axis('user_id', inplace=True)

### Agora, vamos trabalhar algumas features em diferentes tabelas para criar as métricas que vamos utilizar para tentar identificar comportamentos dos usuários no app.

In [ ]:
# Vamos agrupar os jogos por user_id para calcular a quantidade de jogos que
# cada usuário tem cadastrado no app e preparar para concatenar ao dataset principal.
grouped_games = users_games.groupby(by='user_id').count()
grouped_games.rename(columns={'id':'qtde_games'}, inplace=True)
grouped_games.drop(columns=['game_id', 
                            'platform_id', 
                            'network_id', 
                            'is_favorite', 
                            'status_id'], 
                   inplace=True
                  )

In [ ]:
# Vamos fazer o mesmo para a quantidade de Grupos criados ('events').
events.rename(columns={'created_by':'user_id'}, inplace=True)
grouped_events = events.groupby(by='user_id').count()
grouped_events.rename(columns={'id':'qtde_grupos'}, inplace=True)
grouped_events.drop(grouped_events.iloc[:, 1:], inplace = True, axis = 1) 

In [ ]:
# E o mesmo para a quantidade de conexões ('friends').
grouped_friends = friends.groupby(by='user_id').count()
grouped_friends.drop(columns=['id'], inplace=True)
grouped_friends.rename(columns={'friend_id':'qtde_contatos'}, inplace=True)

In [ ]:
# Também para as ações realizadas pelos usuários no app ('users_events'). 
grouped_user_events = users_events.groupby(by='user_id').count()
grouped_user_events.rename(columns={'id':'qtde_eventos'}, inplace=True)
grouped_user_events.drop(grouped_user_events.iloc[:, 1:], inplace = True, axis = 1) 

In [ ]:
# E também para as mensagens de chat trocadas por cada usuário ('chats').
chats_grouped = chats.groupby(by='user_id').count()
chats_grouped.rename(columns={'chat_id':'qtde_mensagens'}, inplace=True)
chats_grouped.drop(chats_grouped.iloc[:, 1:], inplace = True, axis = 1) 

In [ ]:
# Tambem para a quantidade de Plataformas cadastradas ('platforms').
platforms_grouped = platforms.groupby(by='user_id').count()
platforms_grouped.rename(columns={'id':'qtde_plataformas'}, inplace=True)
platforms_grouped.drop(platforms_grouped.iloc[:, 1:], inplace = True, axis = 1) 

In [ ]:
# E, por fim, para a quantidade de logins realizados pelo usuários, o que mostra a frequência de acessos ('logs').
grouped_logs = logs.groupby(by='user_id').count()
grouped_logs.rename(columns={'id':'qtde_acessos'}, inplace=True)
grouped_logs.drop(columns=['login_datetime'], inplace=True)

In [ ]:
# Agora concatenamos todas as features junto com o dataset principal.
users_gg = pd.concat([gg_users, 
                      grouped_games, 
                      grouped_events, 
                      grouped_friends, 
                      grouped_user_events, 
                      chats_grouped, 
                      platforms_grouped, 
                      grouped_logs], 
                      axis = 1
                    )

In [ ]:
# Os dados nulos precisam ser entendidos pelo modelo, então vamos tratá-los como 0, 
# pois significa que o usuário não tem nenhuma daquelas features cadastradas.
users_gg["qtde_games"].fillna(0, inplace=True)
users_gg["qtde_grupos"].fillna(0, inplace=True)
users_gg["qtde_contatos"].fillna(0, inplace=True)
users_gg["qtde_eventos"].fillna(0, inplace=True)
users_gg["qtde_mensagens"].fillna(0, inplace=True)
users_gg["qtde_plataformas"].fillna(0, inplace=True)
users_gg["qtde_acessos"].fillna(0, inplace=True)

In [ ]:
# Eliminando as linhas que não têm informação de last_login. Não serão úteis.
users_gg.dropna(how='any', subset=['last_login'], inplace=True)

In [ ]:
# Ao tentar converter creation_datetime, que é string, para o formato datetime, 
# deu erro porque alguma linha tinha valor '0000-00-00 00:00:00'.
# Precisamos encontrar essa(s) linha(s) e eliminá-la(s).
users_gg.loc[users_gg['creation_datetime']=='0000-00-00 00:00:00']

In [ ]:
users_gg.drop(labels=49211, inplace=True)

In [ ]:
# Agora vamos converter todos os timestamps para datetime.
users_gg['creation_datetime'] = pd.to_datetime(users_gg['creation_datetime'])
users_gg['last_login'] = pd.to_datetime(users_gg['last_login'])
users_gg['last_interaction'] = pd.to_datetime(users_gg['last_interaction'])

# Depois vamos criar uma coluna de churn, com base no número de dias de inatividade 
# entre hoje e a data de último login. Se for maior que 30 dias, consideramos churn.
# Essa coluna será o nosso targets para os modelos.
# Definimos 30 dias de acordo com o que a área de negócios considera um período 
# indicativo de parada de uso do app.
curr_time = pd.to_datetime("now")
users_gg['inatividade'] = curr_time - users_gg['last_interaction']
users_gg['inatividade_days'] = users_gg['inatividade'].dt.days
users_gg['churn'] = users_gg['inatividade_days']>30

In [ ]:
# Agora vamos criar uma coluna que nos diga quanto tempo o usuário passou dentro do app 
# entre o primeiro login 'creation_datetime' e a última interação 'last_interaction'.
# E vamos criar outra coluna identificando aqueles que ficaram menos de 30 dias no app.
users_gg['tempo_uso'] = users_gg['last_interaction'] - users_gg['creation_datetime']
users_gg['tempo_uso_days'] = users_gg['tempo_uso'].dt.days
users_gg['pouco_uso'] = users_gg['tempo_uso_days']<30

In [ ]:
plt.hist('tempo_uso_days', bins=20, data=users_gg)

### Vemos que cerca de 20% dos usuários tiveram menos de 30 dias de uso do app.
### 

### Agora, como a empresa começou a fazer modificações no app e iniciou ações de marketing para atração de novos usuários a partir de Julho de 2020, definimos com a área de negócios que vamos trabalhar com os usuários que entraram a partir dessa data apenas, desconsiderando os usuários antigos.

In [ ]:
gg_dataset = users_gg.loc[users_gg['creation_datetime']>'2020-07-01']

### Analisando as features nesse período, vemos que há muitas sem qualquer valor e uma grande quantidade delas são strings, que precisarão ser convertidas em números para a análise dos modelos.
### Vamos eliminar as nulas e ver como tratar as incompletas.

In [ ]:
gg_dataset.dropna(how='all', axis=1, inplace=True)

In [ ]:
gg_dataset.info()

In [ ]:
# Algumas features não nos serão úteis pois contém informações irrelevantes para as análises.
# Vamos eliminá-las.
gg_dataset = gg_dataset.drop(columns=['pass_hash', 
                                           'uploaded_photo_timestamp', 
                                           'top_image', 
                                           'profile_bg_id', 
                                           'phone_country', 
                                           'update_datetime', 
                                           'remember_token', 
                                           'access_token', 
                                           'special_mark', 
                                           'firebase_id']
                                 )

In [ ]:
# agora, pra facilitar as análises numéricas, vamos transformas algumas colunas em categóricas.
# Por exemplo, vamos considerar apenas a presença ou não de avatar, sem especificar qual.
# Da mesma forma, a presença ou não de ID para as plataformas, sem considerar o ID específico.
gg_dataset['email'] = np.where(gg_dataset['email'].isnull(), 0, 1)
gg_dataset['name'] = np.where(gg_dataset['name'].isnull(), 0, 1)
gg_dataset['nickname'] = np.where(gg_dataset['nickname'].isnull(), 0, 1)
gg_dataset['avatar'] = np.where(gg_dataset['avatar'].isnull(), 0, 1)
gg_dataset['profile_pic_id'] = np.where(gg_dataset['profile_pic_id'].isnull(), 0, 1)
gg_dataset['city'] = np.where(gg_dataset['city'].isnull(), 0, 1)
gg_dataset['ip_country_code'] = np.where(gg_dataset['ip_country_code']=='BR', 1, 0)
gg_dataset['fbid'] = np.where(gg_dataset['fbid'].isnull(), 0, 1)
gg_dataset['quickblox_id'] = np.where(gg_dataset['quickblox_id'].isnull(), 0, 1)
gg_dataset['steam_id'] = np.where(gg_dataset['steam_id'].isnull(), 0, 1)
gg_dataset['psn_id'] = np.where(gg_dataset['psn_id'].isnull(), 0, 1)
gg_dataset['xboxlive_id'] = np.where(gg_dataset['xboxlive_id'].isnull(), 0, 1)
gg_dataset['nintendo_id'] = np.where(gg_dataset['nintendo_id'].isnull(), 0, 1)
gg_dataset['gamecenter_id'] = np.where(gg_dataset['gamecenter_id'].isnull(), 0, 1)
gg_dataset['googleplay_id'] = np.where(gg_dataset['googleplay_id'].isnull(), 0, 1)
gg_dataset['battlenet_id'] = np.where(gg_dataset['battlenet_id'].isnull(), 0, 1)
gg_dataset['uplay_id'] = np.where(gg_dataset['uplay_id'].isnull(), 0, 1)
gg_dataset['origin_id'] = np.where(gg_dataset['origin_id'].isnull(), 0, 1)
gg_dataset['youtube_id'] = np.where(gg_dataset['youtube_id'].isnull(), 0, 1)
gg_dataset['twitch_id'] = np.where(gg_dataset['twitch_id'].isnull(), 0, 1)
gg_dataset['phone_number'] = np.where(gg_dataset['phone_number'].isnull(), 0, 1)

In [ ]:
gg_dataset['lng'] = gg_dataset['lng'].astype('float64')

In [ ]:
gg_dataset.info()

In [ ]:
# Creating a dataframe with 30% registers of original dataframe 
gg_30 = gg_dataset.sample(frac = 0.3) 
  
# Creating dataframe with the rest of the 70% registers 
gg_70 = gg_dataset.drop(gg_30.index) 
  
print("\n30% of the givem DataFrame:") 
print(gg_30.info()) 
  
print("\nrest 70% of the given DataFrame:") 
print(gg_70.info())

In [ ]:
gg_train = gg_70.copy()

In [ ]:
gg_test = gg_30.copy()
gg_test.drop(columns=['churn'], inplace=True)

In [ ]:
gg_train.to_csv('../CSVs/games_train.csv')

In [ ]:
gg_test.to_csv('../CSVs/games_test.csv')